In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

cuda = torch.cuda.is_available()

In [12]:
import torchvision
from PIL import Image

In [18]:
class ConvBlock(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
                          nn.Conv2d(in_channels=C_in, out_channels=C_out, kernel_size=kernel_size, stride=stride),
                          nn.ReLU(),
                          nn.MaxPool2d(2))
        
    def forward(self, x):
        return self.block(x)
    
class BaselineModel(nn.Module):
    def __init__(self, num_blocks):
        super(BaselineModel, self).__init__()
        layers = []
        num_classes = 8
        channels = [1, 64, 128, 256] # this needs to be modified according to num_blocks
        
        for i in range(num_blocks):
            layers.append(ConvBlock(C_in=channels[i], C_out=channels[i+1], kernel_size=5, stride=1))
        
        layers.append(nn.Dropout(p=0.25))
        
        layers.append(nn.Linear(channels[-1], 512))
        
        layers.append(nn.Dropout(p=0.5))
        
        layers.append(nn.Linear(512, num_classes))
        
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x)

In [19]:
MyModel = BaselineModel(num_blocks=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MyModel.parameters(), lr=1e-3, weight_decay=1e-5)
device = torch.device("cuda" if cuda else "cpu")
print(MyModel)

BaselineModel(
  (net): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (2): ConvBlock(
      (block): Sequential(
        (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (3): Dropout(p=0.25)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): Dropout(p=0.5)
    (6): Linear(in_features=512, out_features=8, bias=True)
  )
)


In [14]:
img = Image.open("../cohn-kanade-images/S005/001/S005_001_00000011.png")

In [26]:
resized_img = torchvision.transforms.Resize((224,224))(img)

In [33]:
img_tensor = torchvision.transforms.ToTensor()(resized_img)
img_tensor

tensor([[[0.2039, 0.5922, 0.7294,  ..., 0.0000, 0.0000, 0.0000],
         [0.0706, 0.2157, 0.2549,  ..., 0.0000, 0.0000, 0.0000],
         [0.1882, 0.5176, 0.5804,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.8902, 0.9059, 0.9333,  ..., 0.8353, 0.8353, 0.8353],
         [0.8863, 0.9020, 0.9137,  ..., 0.8353, 0.8431, 0.8471],
         [0.5451, 0.5569, 0.5608,  ..., 0.5137, 0.5176, 0.5216]]])

In [40]:
img_tensor = img_tensor.unsqueeze(dim = 0)

In [41]:
MyModel(img_tensor)

RuntimeError: size mismatch, m1: [6144 x 24], m2: [256 x 512] at /Users/soumith/b101_2/2019_02_08/wheel_build_dirs/wheel_3.7/pytorch/aten/src/TH/generic/THTensorMath.cpp:940